Amazon Kindle Review Sentimental Analysis Project

In [1]:
#Checkig wheather the review is positive or negative 

Best Practices 

1)preprocessing and cleaning 

2)train test split 

3)BOW,TF-IDF,Word2vec 

4)Apply Ml algorithms

In [45]:
# Load the dataset 
import pandas as pd  
df=pd.read_csv("all_kindle_review.csv")

In [46]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [47]:
# For this use case we only need reviewtext and rating

In [48]:
df=df[["reviewText","rating"]]

In [49]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [50]:
#lets do some exploratory data analysis on the data

In [51]:
df.shape

(12000, 2)

In [52]:
# Checking the missing values
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [53]:
# to check the unique ratings
df["rating"].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [54]:
#Checking weather the dataset is balanced or imbalanced
df["rating"].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

Preprocessing And Cleaning

In [55]:
#Lets solve this case in such as way that 
#ratings <=3 are considered as negative review
#ratings >3 are considered as positive review

In [56]:
#positive review is cosidered as 1 and negative review is 0
df["rating"]=df["rating"].apply(lambda x:0 if x<=3 else 1)


In [57]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",0
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,0
3,Aggie is Angela Lansbury who carries pocketboo...,0
4,I did not expect this type of book to be in li...,1


In [58]:
df.rating.value_counts()

rating
0    6000
1    6000
Name: count, dtype: int64

In [59]:
df.rating.unique()

array([0, 1], dtype=int64)

Imporatnt Things to consider while cleaning Corpus

1)Convert to lowercase

text = text.lower()
    
2)Remove URLs, emails, and mentions (@username)

text = re.sub(r'http\S+|www\S+|@\S+', '',text)
    
3)Remove punctuation and special characters

text = re.sub(r'[^\w\s]', '', text)
    
4)Remove numbers

text = re.sub(r'\d+', '', text)
    
5)Remove extra whitespaces

text = ' '.join(text.split())


In [60]:
#Lower all the cases 
df["reviewText"]=df["reviewText"].str.lower()

In [61]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",0
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,0
3,aggie is angela lansbury who carries pocketboo...,0
4,i did not expect this type of book to be in li...,1


In [62]:
#!pip install bs4 
import re 
from bs4 import BeautifulSoup
import nltk 
from nltk.corpus import stopwords

In [63]:
# Removing special characters 
df["reviewText"]=df['reviewText'].apply(lambda x:re.sub('[^a-zA-Z0-9]+',' ',x))
# Removing stop words
df["reviewText"]=df["reviewText"].apply(lambda x:" ".join([y for y in x.split() if y not in set(stopwords.words("english"))]))
 # Remove URLs, emails, and mentions (@username)
df["reviewText"]=df["reviewText"].apply(lambda x:re.sub(r'http\S+|www\S+|@\S+', ' ',x))

In [67]:
#df.head()

In [41]:
import numpy as np 
import sys  
np.set_printoptions(threshold=sys.maxsize)
#print(df["reviewText"])

In [68]:
#for removing extra spaces
df["reviewText"]=df["reviewText"].apply(lambda x: " ".join(x.split()))

In [69]:
#df.head()

Applying Lemmatizer 

In [70]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [71]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [73]:
df["reviewText"]=df["reviewText"].apply(lambda x:lemmatize_words(x))

In [75]:
#df

Train test Split

In [76]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df["reviewText"],df["rating"],test_size=0.20,random_state=42)

In [83]:
X_train.shape,y_train.shape

((9600,), (9600,))

In [82]:
X_test.shape,y_test.shape

((2400,), (2400,))

Applying BOW

In [84]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()

In [89]:
#for training data
X_train_bow=bow.fit_transform(X_train).toarray()
# for test data
X_test_bow=bow.transform(X_test).toarray()

Applying TF-IDF

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()

In [90]:
#for training data
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
# for test data
X_test_tfidf=tfidf.transform(X_test).toarray()

Applying ML Algorithms(Naive bayes)

In [91]:
from sklearn.naive_bayes import GaussianNB
nb_bow_model=GaussianNB()

In [92]:
nb_bow_model.fit(X_train_bow,y_train)

GaussianNB()

In [94]:
nb_tfidf_model=GaussianNB()
nb_tfidf_model.fit(X_train_tfidf,y_train)

GaussianNB()

Making Predictions

In [96]:
y_pred_bow=nb_bow_model.predict(X_test_bow)
y_pred_tfidf=nb_tfidf_model.predict(X_test_tfidf)

Evaluating the Model

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report

BOW model Perfomance 

In [99]:
print("BOW MODEL PERFOMANCE ")
print("-----------------------")
print("Accuracy: ", accuracy_score(y_test,y_pred_bow))
print("Precision: ", precision_score(y_test,y_pred_bow))
print("\n")
print(confusion_matrix(y_test,y_pred_bow))
print(classification_report(y_test,y_pred_bow))


BOW MODEL PERFOMANCE 
-----------------------
Accuracy:  0.59
Precision:  0.6426767676767676


[[907 283]
 [701 509]]
              precision    recall  f1-score   support

           0       0.56      0.76      0.65      1190
           1       0.64      0.42      0.51      1210

    accuracy                           0.59      2400
   macro avg       0.60      0.59      0.58      2400
weighted avg       0.60      0.59      0.58      2400



TF-IDF MODEL PERFOMANCE

In [100]:
print("TF-IDF MODEL PERFOMANCE ")
print("-----------------------")
print("Accuracy: ", accuracy_score(y_test,y_pred_tfidf))
print("Precision: ", precision_score(y_test,y_pred_tfidf))
print("\n")
print(confusion_matrix(y_test,y_pred_tfidf))
print(classification_report(y_test,y_pred_tfidf))


TF-IDF MODEL PERFOMANCE 
-----------------------
Accuracy:  0.63875
Precision:  0.6511013215859031


[[794 396]
 [471 739]]
              precision    recall  f1-score   support

           0       0.63      0.67      0.65      1190
           1       0.65      0.61      0.63      1210

    accuracy                           0.64      2400
   macro avg       0.64      0.64      0.64      2400
weighted avg       0.64      0.64      0.64      2400



In [101]:
#there is a little bit increase in accuracy of Tf-Idf compare to BOW 
# for more accuracy you can also try using Word2vec